In [5]:
# pip install sentence-transformers faiss-cpu
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, os, json, re

# 1) модель (выберите одну)
model_name = "intfloat/multilingual-e5-base"   # или "BAAI/bge-m3", "paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

def build_chunk_text(title: str, body: str, title_weight: int = 3) -> str:
    """
    Усиливаем заголовок: повторяем его несколько раз + помечаем как title.
    Для E5/INSTRUCT-моделей хорошо добавлять префиксы 'passage:'
    """
    title_clean = re.sub(r"\s+", " ", title).strip()
    body_clean  = re.sub(r"\s+", " ", body).strip()
    boosted = ("title: " + title_clean + " ") * title_weight
    return f"passage: {boosted}content: {body_clean}"

# 2) парсим data.md на чанки
def parse_md_chunks(filepath):
    with open(filepath, encoding="utf-8") as f:
        content = f.read()
    raw_chunks = content.split("---")
    docs = []
    for i, chunk in enumerate(raw_chunks):
        chunk = chunk.strip()
        if not chunk:
            continue
        # Ищем первый заголовок # ...
        m = re.match(r"#\s*(.+?)(?:\n|$)(.*)", chunk, re.DOTALL)
        if m:
            title = m.group(1).strip()
            body = m.group(2).strip()
        else:
            title = f"chunk_{i+1:03}"
            body = chunk
        text = build_chunk_text(title, body)
        docs.append({"id": f"chunk_{i+1:03}", "text": text})
    return docs

docs = parse_md_chunks("data.md")

# 3) эмбеддинги (батчево)
texts = [d["text"] for d in docs]
emb = model.encode(texts, batch_size=64, show_progress_bar=True, normalize_embeddings=True)  # normalize=True -> косинус = dot

emb = np.asarray(emb, dtype="float32")
dim = emb.shape[1]

# 4) FAISS индекс (Inner Product для косинуса, так как мы нормировали)
index = faiss.IndexFlatIP(dim)
index.add(emb)

# 5) сохраняем индекс и метаданные
faiss.write_index(index, "smartwop_docs.faiss")
with open("smartwop_docs_meta.json", "w", encoding="utf-8") as f:
    json.dump(docs, f, ensure_ascii=False, indent=2)

print("Готово:", index.ntotal, "векторов")


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Готово: 107 векторов
